In [1]:
import pandas as pd
import numpy as np
import os
import sys
import Bio
import epitopepredict as ep

In [2]:
train1_sample=pd.read_csv(os.path.join(sys.path[0], "Analysis/train1_sample.csv"), index_col=False)
print(train1_sample.shape)

test1_sample=pd.read_csv(os.path.join(sys.path[0], "Analysis/test1_sample.csv"), index_col=False)
print(test1_sample.shape)

(87052, 8)
(21535, 8)


In [3]:
blosum = ep.blosum50

def blosum_encode(seq):
    #encode a peptide into blosum features
    s=list(seq)
    x = pd.DataFrame([blosum[i] for i in seq]).reset_index(drop=True)
    # show_matrix(x)
    e = x.values.flatten()    
    return e

# Training

In [20]:
def blosum_encode_data(train_holdout_df, include_AA, exclude_AA, all_feautre):
    # create blosum encoding for MHC
    blosum50 = train_holdout_df.drop_duplicates(subset=["%s" %(include_AA)], keep="last").drop(columns=["%s" %(exclude_AA), "%s_ID" %(exclude_AA)])
    print(blosum50.shape)
    X = blosum50["%s" %(include_AA)].apply(lambda x: pd.Series(blosum_encode(x)),1)
    print(X.shape)

    X = X.add_suffix('_%s' %(include_AA))

    

    blosum50_2 = pd.concat([blosum50[["%s"%(include_AA), "%s_ID"%(include_AA)]], X], axis=1).reset_index(drop=True)
    print(blosum50_2.shape)
    # return blosum50_2

    blosum50_3 = blosum50_2.drop(columns=["%s_ID" %(include_AA)])
    
    cols=["%s" %(include_AA), "%s_ID" %(include_AA), "Pair_ID"]
    all_feautre_2 = all_feautre[cols]
    blosum50_4 = all_feautre_2.merge(blosum50_3, on=["%s" %(include_AA)], how="left")
    print(blosum50_4.shape)

    return blosum50_2

In [18]:
# train
train1_MHC_Unique, train1_MHC = blosum_encode_data(train1_sample, "MHC", "Peptide", train1_sample)
train1_Peptide_Unique, train1_Peptide = blosum_encode_data(train1_sample, "Peptide", "MHC", train1_sample)

(51, 6)
(51, 850)
(51, 852)
(87052, 853)
(9611, 6)
(9611, 375)
(9611, 377)
(87052, 378)


In [ ]:
# save MHC and peptide bolusm encoding separately
train1_MHC_Unique.to_csv(os.path.join(sys.path[0],"Analysis/train1_MHC_Unique.csv"), index=False)
train1_Peptide_Unique.to_csv(os.path.join(sys.path[0],"Analysis/train1_Peptide_Unique.csv"), index=False)

In [21]:
# test
test1_MHC_Unique = blosum_encode_data(test1_sample, "MHC", "Peptide", test1_sample)
test1_Peptide_Unique = blosum_encode_data(test1_sample, "Peptide", "MHC", test1_sample)

(49, 6)
(49, 850)
(49, 852)
(21535, 853)
(2778, 6)
(2778, 375)
(2778, 377)
(21535, 378)


In [27]:
# save MHC and peptide bolusm encoding separately
test1_MHC_Unique.to_csv(os.path.join(sys.path[0],"Analysis/test1_MHC_Unique.csv"), index=False)
test1_Peptide_Unique.to_csv(os.path.join(sys.path[0],"Analysis/test1_Peptide_Unique.csv"), index=False)

In [23]:
# check null
test1_Peptide_Unique.isnull().sum().sum()

0

### merge MHC and peptide blosum encoding

In [15]:
# train data

train1_MHC = train1_sample.merge(train1_MHC_Unique.drop(columns=["MHC_ID"]), how="left", on=["MHC"])
print(train1_sample.shape, train1_MHC.shape)

train1_MHC_Peptide = train1_MHC.merge(train1_Peptide_Unique.drop(columns=["Peptide_ID"]), how="left", on=["Peptide"])
print(train1_MHC.shape, train1_MHC_Peptide.shape)

train1_MHC_Peptide.to_csv(os.path.join(sys.path[0],"Analysis/blosum50_train.csv"), index=False)

(87052, 8) (87052, 858)
(87052, 858) (87052, 1233)


In [28]:
# test data

test1_MHC = test1_sample.merge(test1_MHC_Unique.drop(columns=["MHC_ID"]), how="left", on=["MHC"])
print(test1_sample.shape, test1_MHC.shape)

test1_MHC_Peptide = test1_MHC.merge(test1_Peptide_Unique.drop(columns=["Peptide_ID"]), how="left", on=["Peptide"])
print(test1_MHC.shape, test1_MHC_Peptide.shape)

test1_MHC_Peptide.to_csv(os.path.join(sys.path[0],"Analysis/blosum50_test.csv"), index=False)

(21535, 8) (21535, 858)
(21535, 858) (21535, 1233)
